## Writing a Simple Action Server using the Execute Callback
This tutorial covers using the simple_action_server library to create a Fibonacci action server in Python. This example action server generates a Fibonacci sequence, the goal is the order of the sequence, the feedback is the sequence as it is computed, and the result is the final sequence.

If the order of the sequence is greater than 100, the action is aborted by the server.

In addition, the action can be cancelled by the client at any time during the execution.

In [ ]:
import rospy
from actionlib import SimpleActionServer
from actionlib_tutorials.msg import FibonacciFeedback, FibonacciResult, \
    FibonacciAction

Here we import the `SimpleActionServer` class from the `actionlib` library, and the  classes for the messages. The action specification generates such messages for sending goals, receiving feedbacks, etc...

In [ ]:
action_name = 'fibonacci'
feedback = FibonacciFeedback()
result = FibonacciResult()

This is the name of the action, and the objects for storing the feedback and result data.

In [ ]:
def execute_cb(goal):
    r = rospy.Rate(1)
    success = True
    feedback.sequence = []
    feedback.sequence.append(0)
    feedback.sequence.append(1)
    if goal.order > 100:
        result.sequence = feedback.sequence
        rospy.loginfo('%s: Aborted' % action_name)
        action_server.set_aborted(result, "Sequence aborted due to excessive order")
        return
    
    rospy.loginfo('%s: Executing, creating fibonacci sequence of order %i with seeds %i, %i' % 
                  (action_name, goal.order, feedback.sequence[0], feedback.sequence[1]))
    for i in range(1, goal.order):
            if not action_server.is_preempt_requested():
                feedback.sequence.append(feedback.sequence[i] + feedback.sequence[i-1])
                action_server.publish_feedback(feedback)
                r.sleep()

    if not action_server.is_preempt_requested():
        result.sequence = feedback.sequence
        rospy.loginfo('%s: Succeeded' % action_name)
        action_server.set_succeeded(result, "Sequence completed successfully")

This is the execute callback function that we'll run everytime a new goal is received.

An important component of an action server is the ability to allow an action client to request that the goal under execution be canceled. When a client requests that the current goal be preempted, the action server should cancel the goal, perform any necessary cleanup, and call the `set_preempted` function, which signals that the action has been preempted by user request. Here, we'll check if we've been preempted every second. We could, alternatively, receive a callback when a preempt request is received.

The Fibonacci sequence is put into the feedback variable and then published on the feedback channel provided by the action server. Then, the action continues looping and publishing feedback.

Once the action has finished computing the Fibonacci sequence, the action server notifies the action client that the goal is complete by calling `set_succeeded`.

In [ ]:
def preempt_cb():
    result.sequence = feedback.sequence
    rospy.loginfo('%s: Preempted' % action_name)
    action_server.set_preempted(result, "Sequence preempted")
    return

In [ ]:
action_server = SimpleActionServer(action_name, FibonacciAction, 
                    execute_cb = execute_cb, auto_start = False)

In [ ]:
action_server.register_preempt_callback(preempt_cb)

In [ ]:
rospy.init_node('fibonacci')
action_server.start()
rospy.spin()

[Writing a Simple Action Client](Client.ipynb).